In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Mon May 15 07:33:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-eaa_mn7z/farm-haystack_a541388af1114af5ba2ecf491272b77b
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-eaa_mn7z/farm-haystack_a541388af1114af5ba2ecf491272b77b
  Resolved https://github.com/deepset-ai/haystack.git to commit 6eb251d1f09a2cbc38ceb41abc085dc19d71a660
  Installing build dependencies ... done
  Getting requirements to

In [ ]:
from haystack.nodes import FARMReader


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing the model from hugging face for the SQuAD dataset.

In [ ]:

reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)
data_dir = "data/"
# data_dir = "PATH/TO_YOUR/TRAIN_DATA"


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


#### Training the model with our custom dataset

In [ ]:
reader.train(data_dir=data_dir, train_filename="/content/drive/MyDrive/train.json", use_gpu=True, n_epochs=10, save_dir="my_model",)

Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

...

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

#### Saving model to the directory

In [ ]:
# Saving the model happens automatically at the end of training into the `save_dir` you specified
# However, you could also save a reader manually again via:
reader.save(directory="my_model")

In [ ]:
# If you want to load it at a later point, just do:
new_reader = FARMReader(model_name_or_path="my_model")

### Evaluating the model with the test dataset

In [ ]:
reader_eval_results = new_reader.eval_on_file("data/", "/content/drive/MyDrive/test.json", device="cuda")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.


Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
reader_eval_results

{'EM': 23.595505617977526,
 'f1': 52.09389053824669,
 'top_n_accuracy': 82.02247191011236,
 'top_n': 4,
 'EM_text_answer': 16.25,
 'f1_text_answer': 47.95445322379944,
 'top_n_accuracy_text_answer': 80.0,
 'top_n_EM_text_answer': 17.5,
 'top_n_f1_text_answer': 51.603547364379644,
 'Total_text_answer': 80,
 'EM_no_answer': 88.88888888888889,
 'f1_no_answer': 88.88888888888889,
 'top_n_accuracy_no_answer': 100.0,
 'Total_no_answer': 9}

In [ ]:
context = '''Los Angeles is a sprawling Southern California city and the center of the nation’s film and television industry. Near its iconic Hollywood sign, studios such as Paramount Pictures, Universal and Warner Brothers offer behind-the-scenes tours. On Hollywood Boulevard, TCL Chinese Theatre displays celebrities’ hand- and footprints, the Walk of Fame honors thousands of luminaries and vendors sell maps to stars’ homes'''

In [ ]:
context

'Los Angeles is a sprawling Southern California city and the center of the nation’s film and television industry. Near its iconic Hollywood sign, studios such as Paramount Pictures, Universal and Warner Brothers offer behind-the-scenes tours. On Hollywood Boulevard, TCL Chinese Theatre displays celebrities’ hand- and footprints, the Walk of Fame honors thousands of luminaries and vendors sell maps to stars’ homes'

In [ ]:
new_reader.predict_on_texts("What are some iconic thing about the city?",[context])

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'What are some iconic thing about the city?',
 'no_ans_gap': 19.617967128753662,
 'answers': [<Answer {'answer': 'Paramount Pictures, Universal and Warner Brothers', 'type': 'extractive', 'score': 0.8572891354560852, 'context': 'y. Near its iconic Hollywood sign, studios such as Paramount Pictures, Universal and Warner Brothers offer behind-the-scenes tours. On Hollywood Boule', 'offsets_in_document': [{'start': 161, 'end': 210}], 'offsets_in_context': [{'start': 51, 'end': 100}], 'document_ids': ['cffc37319f1f9dad8d5b8087ccfb7bdb'], 'meta': {}}>]}

###Inference Using Pipeline


In [ ]:
from haystack import Pipeline, Document
from haystack.utils import print_answers
# reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
p = Pipeline()
p.add_node(component=new_reader, name="Reader", inputs=["Query"])
res = p.run(
    query="What is the historic importance of the city?", documents=[Document(content=context)]
)
print_answers(res,details="medium")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What is the historic importance of the city?'
'Answers:'
[   {   'answer': 'the center of the nation’s film and television industry',
        'context': 'les is a sprawling Southern California city and the center '
                   'of the nation’s film and television industry. Near its '
                   'iconic Hollywood sign, studios such ',
        'score': 0.46960556507110596}]
